In [4]:
#score:0.81649

import numpy as np
import pandas as pd
df = pd.read_csv('ShanghaiPMI25_tr.csv')

#deal with outlier
#outlier_idx = np.where(df['PM_Jingan'].values > 300)[0]
#df.drop(df.index[outlier_idx],inplace=True)

y = df['PM_Jingan'].values
mean = y.mean()
std = y.std()
print(mean, std)
upper_bound = mean + 0.5 * std

outlier_idx = np.where(df['PM_Jingan'].values > upper_bound)[0]
df.drop(df.index[outlier_idx],inplace=True)
y = df['PM_Jingan'].values
print(len(outlier_idx))

df.drop(['No','year','PM_Jingan',],axis=1,inplace=True)

cbwd = pd.get_dummies(df['cbwd'],prefix='cbwd')

#'DEWP','precipitation','Iprec','day','hour'
season = pd.get_dummies(df['season'],prefix='season')
df = pd.concat([df,season,cbwd],axis=1)
df.drop(['season','season_1','cbwd','cbwd_cv'],axis=1,inplace=True)
X = df.values
df.head()

59.9254785779 52.4084201428
2646


,month,day,hour,DEWP,HUMI,PRES,TEMP,Iws,precipitation,Iprec,season_2,season_3,season_4,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_SW
0,1,1,0,-5,69.00,1023,0,60,0.0,0.0,0,0,1,0,0,0,1
1,1,1,1,-5,69.00,1023,0,62,0.0,0.0,0,0,1,0,0,0,1
2,1,1,3,-4,74.41,1023,0,65,0.0,0.0,0,0,1,0,0,0,1
3,1,1,4,-4,80.04,1023,-1,68,0.0,0.0,0,0,1,0,0,0,1
4,1,1,5,-4,80.04,1023,-1,70,0.0,0.0,0,0,1,0,0,0,1


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
rs.fit(X_train)
X_train_std = rs.transform(X_train)
X_test_std = rs.transform(X_test)

#train y
third_per,first_per = np.percentile(y_train,[75,25])
rs_y = RobustScaler()
rs_y.fit(y_train.reshape(-1, 1))
y_train = rs_y.transform(y_train.reshape(-1, 1))

from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier.fit(X_train_std, y_train)
print(classifier.coef_)
y_pred = classifier.predict(X_test_std)
y_pred_trans = y_pred * (third_per - first_per) + first_per
print(mean_squared_error(y_test,y_pred_trans))
rs_y.transform(y_test.reshape(-1, 1))
print(classifier.score(X_test_std, y_test))


[[ 0.14991191  0.06097688  0.05194787  0.39375818 -0.23872761 -0.19193841
  -0.35679926 -0.13412842 -0.02713801 -0.00540922 -0.53050777 -0.51518254
   0.16294686 -0.52883976 -0.02360243 -0.50627197 -0.10573382]]
999.50558373
-2.53440586267


In [5]:
#train all the data

from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
rs.fit(X)
X_std = rs.transform(X)

third_per,first_per = np.percentile(y,[75,25])
y = y.reshape(-1,1)
rs_y = RobustScaler()
rs_y.fit(y)
y = rs_y.transform(y)

from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier.fit(X_std, y)
classifier.coef_

array([[ 0.11370436,  0.01613668,  0.03960737,  0.15956225, -0.14449087,
        -0.15083023, -0.20824464, -0.10720902, -0.01027641, -0.00630001,
        -0.37040943, -0.40235796, -0.03596603, -0.43805036, -0.01792612,
        -0.34626896,  0.01010978]])

In [6]:
#get the input
test_data = pd.read_csv("ShanghaiPMI25_te.csv")
my_no = np.matrix(test_data['No'])
#the_ones = np.ones_like(my_no.T)
my_no = np.squeeze(np.asarray(my_no))

test_data.drop(['No','year',],axis=1,inplace=True)
#'DEWP','precipitation','Iprec','day','hour',
test_season = pd.get_dummies(test_data['season'],prefix='season')

test_cbwd = pd.get_dummies(test_data['cbwd'],prefix='cbwd')

test_data = pd.concat([test_data,test_season,test_cbwd],axis=1)
test_data.drop(['season','season_1','cbwd','cbwd_cv'],axis=1,inplace=True)
#print(test_data)
print(test_data.head())
my_X = test_data.values


my_X = rs.transform(my_X)
#my_x = np.hstack((the_ones,test_matrix_std[:,0:test_matrix_std.shape[1]]))

my_y = classifier.predict(my_X)
my_y = my_y * (third_per - first_per) + first_per
result = np.squeeze(np.asarray(my_y))

#result -= 20
#reuslt = abs(result)
result[result<0]=abs(result[result<0])
#print(result)
print(np.mean(result))
final_result = pd.DataFrame({'No' : my_no, 'PM_Jingan' : result})
final_result.to_csv("last_try_submission_delete_2std_adding_dewp_preci_Iprec_year_hour_get_cbwd_move_halfstd.csv", index=False)

   month  day  hour  DEWP   HUMI  PRES  TEMP  Iws  precipitation  Iprec  \
0      9   19     4    16  77.79  1013    20    0            0.0    0.0   
1      9   19     6    18  93.94  1013    19    5            0.3    0.3   
2      9   19     7    17  88.20  1014    19    3            2.2    2.5   
3      9   19     8    18  93.94  1014    19    7            1.5    4.0   
4      9   19     9    18  93.94  1014    19    9            1.2    5.2   

   season_2  season_3  season_4  cbwd_NE  cbwd_NW  cbwd_SE  cbwd_SW  
0         0         1         0        0        0        0        0  
1         0         1         0        0        1        0        0  
2         0         1         0        1        0        0        0  
3         0         1         0        1        0        0        0  
4         0         1         0        1        0        0        0  
26.8836465863
